In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, balanced_accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.feature_selection import mutual_info_regression
from sklearn.utils import resample
import os

**Імпорт даних**

In [41]:
file_train_path = '/kaggle/input/final-project-dataset/final_proj_data.csv'
file_test_path = '/kaggle/input/final-project-dataset/final_proj_test.csv'

train_data = pd.read_csv(file_train_path)
valid_data = pd.read_csv(file_test_path)

print("Дані для навчання")
print(train_data.head())

print(f"Кількість рядків у тренувальних даних: {train_data.shape[0]}")
print()
print("Дані для валідації")
print(valid_data.head())

print(f"Кількість рядків у валідаційних даних: {valid_data.shape[0]}")

print("==========")
# Підрахунок кількості значень для цільової змінної 'y'
class_counts = train_data['y'].value_counts()

print("Кількість кожного класу в тренувальних даних:")
print(class_counts)

Дані для навчання
   Var1  Var2  Var3  Var4  Var5    Var6  Var7  Var8  Var9  Var10  ...  \
0   NaN   NaN   NaN   NaN   NaN   812.0  14.0   NaN   NaN    NaN  ...   
1   NaN   NaN   NaN   NaN   NaN  2688.0   7.0   NaN   NaN    NaN  ...   
2   NaN   NaN   NaN   NaN   NaN  1015.0  14.0   NaN   NaN    NaN  ...   
3   NaN   NaN   NaN   NaN   NaN   168.0   0.0   NaN   NaN    NaN  ...   
4   NaN   NaN   NaN   NaN   NaN    14.0   0.0   NaN   NaN    NaN  ...   

    Var222      Var223  Var224  Var225  Var226  Var227            Var228  \
0  catzS2D  jySVZNlOJy     NaN    xG3x    Aoh3    ZI9m     ib5G6X1eUxUn6   
1  i06ocsg  LM8l689qOp     NaN    kG3k    WqMG    RAYp           55YFVY9   
2  P6pu4Vl  LM8l689qOp     NaN    kG3k    Aoh3    ZI9m  R4y5gQQWY8OodqDV   
3  BNrD3Yd  LM8l689qOp     NaN     NaN    FSa2    RAYp     F2FyR07IdsN7I   
4  3B1QowC  LM8l689qOp     NaN     NaN    WqMG    RAYp     F2FyR07IdsN7I   

   Var229  Var230  y  
0    mj86     NaN  0  
1    mj86     NaN  0  
2    am7c     NaN

In [42]:
# Розділити дані на два класи
data_class_0 = train_data[train_data['y'] == 0]
data_class_1 = train_data[train_data['y'] == 1]

# Undersample клас 0
data_class_0_undersampled = resample(data_class_0,
                                     replace=False,
                                     n_samples=len(data_class_1),
                                     random_state=42)

# Об'єднати знову два класи
train_data = pd.concat([data_class_0_undersampled, data_class_1])

# Перевірити новий баланс класів
print(train_data['y'].value_counts())

y
0    1305
1    1305
Name: count, dtype: int64


**Визначення кількості пропусків**

In [43]:
def print_missing_values(data):
    # Підрахунок кількості пропусків у кожній колонці після очистки
    missing_values = data.isnull().sum()

    for column, num_missing in missing_values.items():
        print(f"{column} - {num_missing}")


print("навчальні дані:")
print_missing_values(train_data)

навчальні дані:
Var1 - 2571
Var2 - 2555
Var3 - 2555
Var4 - 2545
Var5 - 2545
Var6 - 245
Var7 - 251
Var8 - 2610
Var9 - 2571
Var10 - 2545
Var11 - 2555
Var12 - 2581
Var13 - 251
Var14 - 2555
Var15 - 2610
Var16 - 2545
Var17 - 2545
Var18 - 2545
Var19 - 2545
Var20 - 2610
Var21 - 245
Var22 - 224
Var23 - 2545
Var24 - 379
Var25 - 224
Var26 - 2545
Var27 - 2545
Var28 - 224
Var29 - 2571
Var30 - 2571
Var31 - 2610
Var32 - 2610
Var33 - 2570
Var34 - 2555
Var35 - 224
Var36 - 2555
Var37 - 2545
Var38 - 224
Var39 - 2610
Var40 - 2555
Var41 - 2571
Var42 - 2610
Var43 - 2555
Var44 - 224
Var45 - 2591
Var46 - 2555
Var47 - 2571
Var48 - 2610
Var49 - 2555
Var50 - 2571
Var51 - 2452
Var52 - 2610
Var53 - 2571
Var54 - 2555
Var55 - 2610
Var56 - 2579
Var57 - 0
Var58 - 2571
Var59 - 2575
Var60 - 2545
Var61 - 2570
Var62 - 2581
Var63 - 2571
Var64 - 2596
Var65 - 251
Var66 - 2571
Var67 - 2545
Var68 - 2555
Var69 - 2545
Var70 - 2545
Var71 - 2568
Var72 - 1209
Var73 - 0
Var74 - 251
Var75 - 2555
Var76 - 224
Var77 - 2571
Var78 - 224


**Видалення даних з великими пропусками**

In [44]:
def clean_data(data):
    # Визначимо пороги
    column_threshold = 0.8  # Видалити колонки, де більше 80% пропусків
    row_threshold = 0.5  # Видалити рядки, де більше 50% пропусків

    # Підрахунок кількості пропусків у кожній колонці
    missing_values = data.isnull().sum()

    # Видалимо колонки, які мають більше ніж column_threshold пропусків
    columns_to_drop = missing_values[missing_values > column_threshold].index
    data = data.drop(columns=columns_to_drop, errors='ignore')

    print(f"Видалено колонки: {list(columns_to_drop)}")

    # Видалення рядків
    # Обчислимо відсоток пропусків для кожного рядка
    missing_data_rows = data.isnull().mean(axis=1)

    # Видалимо рядки, які мають більше ніж row_threshold пропусків
    rows_to_drop = missing_data_rows[missing_data_rows > row_threshold].index
    data = data.drop(index=rows_to_drop)

    print(f"Видалено рядки: {len(rows_to_drop)}")

    # Виведення розміру очищених даних
    print(f"Розмір очищених даних: {data.shape}")

    return data

train_data = clean_data(train_data)
valid_data = clean_data(valid_data)

print("навчальні дані:")
print_missing_values(train_data)

print("\n====================\n")
print("тестові дані:")
print_missing_values(valid_data)

Видалено колонки: ['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8', 'Var9', 'Var10', 'Var11', 'Var12', 'Var13', 'Var14', 'Var15', 'Var16', 'Var17', 'Var18', 'Var19', 'Var20', 'Var21', 'Var22', 'Var23', 'Var24', 'Var25', 'Var26', 'Var27', 'Var28', 'Var29', 'Var30', 'Var31', 'Var32', 'Var33', 'Var34', 'Var35', 'Var36', 'Var37', 'Var38', 'Var39', 'Var40', 'Var41', 'Var42', 'Var43', 'Var44', 'Var45', 'Var46', 'Var47', 'Var48', 'Var49', 'Var50', 'Var51', 'Var52', 'Var53', 'Var54', 'Var55', 'Var56', 'Var58', 'Var59', 'Var60', 'Var61', 'Var62', 'Var63', 'Var64', 'Var65', 'Var66', 'Var67', 'Var68', 'Var69', 'Var70', 'Var71', 'Var72', 'Var74', 'Var75', 'Var76', 'Var77', 'Var78', 'Var79', 'Var80', 'Var81', 'Var82', 'Var83', 'Var84', 'Var85', 'Var86', 'Var87', 'Var88', 'Var89', 'Var90', 'Var91', 'Var92', 'Var93', 'Var94', 'Var95', 'Var96', 'Var97', 'Var98', 'Var99', 'Var100', 'Var101', 'Var102', 'Var103', 'Var104', 'Var105', 'Var106', 'Var107', 'Var108', 'Var109', 'Var110', 'Var111

**Видалення рядків-дублів**

In [45]:
def drop_duplicates(data):
  data = data.drop_duplicates()

  # розмір очищених даних
  print(f"Розмір очищених даних: {data.shape}")

drop_duplicates(train_data)
drop_duplicates(valid_data)

Розмір очищених даних: (2610, 22)
Розмір очищених даних: (2500, 21)


**Визначення переліку дискретних ознак**

In [46]:
# Визначення переліку дискретних ознак
discrete_features = []

for column in train_data.columns:
    if train_data[column].dtype == 'object':
        discrete_features.append(column)
    elif train_data[column].dtype in ['int64', 'int32'] or (train_data[column].nunique() < 10):
        discrete_features.append(column)

# Видалення 'y' з переліку дискретних ознак, якщо він там присутній
if 'y' in discrete_features:
    discrete_features.remove('y')

print("кількість дискретних ознак: ", len(discrete_features))
print("Перелік дискретних ознак:", discrete_features)

кількість дискретних ознак:  19
Перелік дискретних ознак: ['Var73', 'Var193', 'Var195', 'Var196', 'Var198', 'Var199', 'Var202', 'Var204', 'Var207', 'Var210', 'Var211', 'Var212', 'Var216', 'Var220', 'Var221', 'Var222', 'Var226', 'Var227', 'Var228']


**Розрахунок показника взаємної інформації**

In [47]:
# Кодування категоріальних ознак
autos_encoded = train_data.copy()
for col in discrete_features:
    autos_encoded[col], _ = autos_encoded[col].factorize()

# Визначення вхідних ознак та цільової змінної
X = autos_encoded.drop(columns=['y'])
y = autos_encoded['y']

# Отримання індексів дискретних ознак
discrete_feature_indices = [X.columns.get_loc(col) for col in discrete_features]

# Розрахунок показника взаємної інформації
mi_scores = mutual_info_regression(X, y, discrete_features=discrete_feature_indices)
mi_scores = pd.Series(mi_scores, name='MI Scores', index=X.columns).sort_values(ascending=False)

print("Показники взаємної інформації")
print(mi_scores)

Показники взаємної інформації
Var222    1.146380
Var220    1.146380
Var198    1.146380
Var202    1.109093
Var199    0.596301
Var216    0.417295
Var212    0.162200
Var228    0.147394
Var73     0.141011
Var193    0.100410
Var227    0.074048
Var207    0.070357
Var221    0.063434
Var226    0.019035
Var195    0.009536
Var210    0.007016
Var113    0.006682
Var57     0.005527
Var211    0.005489
Var204    0.000000
Var196    0.000000
Name: MI Scores, dtype: float64


**Відбір ознак за показником MI**

In [48]:
# Створення списку ознак
features = ['Var222', 'Var220', 'Var198', 'Var202', 'Var199', 'Var216', 'Var212', 'Var228', 'Var73']

# Вибір ознак для тренувального датасету
train_data = train_data[features + ['y']]
valid_data = valid_data[features]

# Перевірка виводу
print(train_data.head())

       Var222   Var220   Var198 Var202      Var199   Var216         Var212  \
357   mhe7j0h  1nA68SY  MErO9Nm   hzZb     hJAU4J2  kZJyVg2        NhsEn4L   
5668  RsCQLlz  oatipLp  3FCrXBz   0062     PaagavI  NGZiGpF           leSl   
9685  RlTh9x_  OYHoGfM  FJDupBg   gc6M  lGX0Z38RhO  beKMbmK     E1aAZ0x7vd   
7100  ERg2fT4  OBFOw16  Gs9vcTJ   P8e6     V_KvNzO  IoI4mKe  h0lfDKh52u4GP   
6755  DHPNgqU  XbZitea  0Vr7wZ4   TLWi  yKwwphvwXY  TDcEYQ6     FMSzZ91zL2   

                      Var228  Var73  y  
357            F2FyR07IdsN7I     32  0  
5668           xwM2aC7IdeMC0     24  0  
9685     WfJYmPMksSqa1pajvfG     64  0  
7100                 55YFVY9    128  0  
6755  SbOd7O8ky1wGNxp0Arj0Xs     44  0  


**Кодування категоріальних ознак**

In [49]:
# Застосування One-Hot Encoding до тренувальних даних
train_data_encoded = pd.get_dummies(train_data, columns=features)

# Застосування One-Hot Encoding до валідаційних даних
valid_data_encoded = pd.get_dummies(valid_data, columns=features)

# Переконуємося, що обидва датафрейми мають однакові колонки після One-Hot Encoding
train_columns = set(train_data_encoded.columns) - {'y'}
valid_columns = set(valid_data_encoded.columns)

# Знаходимо стовпці, які є в одному датафреймі, але відсутні в іншому
missing_from_valid = train_columns - valid_columns
missing_from_train = valid_columns - train_columns

# Ініціалізація словника для нових стовпців, що додаються з значенням 0
missing_columns_valid = {col: pd.Series(0, index=valid_data_encoded.index) for col in missing_from_valid}
missing_columns_train = {col: pd.Series(0, index=train_data_encoded.index) for col in missing_from_train}

# Додавання стовпців за допомогою pd.concat
valid_data_encoded = pd.concat([valid_data_encoded, pd.DataFrame(missing_columns_valid)], axis=1)
train_data_encoded = pd.concat([train_data_encoded, pd.DataFrame(missing_columns_train)], axis=1)

# Забезпечення однакового порядку стовпців
valid_data_encoded = valid_data_encoded[train_data_encoded.columns.drop('y')]

# Оновлення основних датасетів
train_data = train_data_encoded
valid_data = valid_data_encoded

**Розподіл на навчальну та тестову вибірки та балансування класів**

In [50]:
train_data_processed = train_data
valid_data_processed = valid_data
# Отримання цільової змінної
X = train_data_processed.drop(columns=['y'])
y = train_data_processed['y']

# Ділимо дані на навчальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ініціалізація SMOTE
sm = SMOTE(random_state=42, k_neighbors=7)

# Застосування SMOTE до навчального набору даних
X_train_resampled, y_train_resampled = sm.fit_resample(X_train, y_train)

print(f"Розмір навчальної вибірки до балансування: {X_train.shape}")
print(f"Розмір навчальної вибірки після балансування: {X_train_resampled.shape}")

# Підрахунок кількості зразків для кожного класу після SMOTE
class_distribution = y_train_resampled.value_counts()

# Виведення результату
print("Розподіл класів після SMOTE:")
print(class_distribution)

Розмір навчальної вибірки до балансування: (2088, 9281)
Розмір навчальної вибірки після балансування: (2092, 9281)
Розподіл класів після SMOTE:
y
0    1046
1    1046
Name: count, dtype: int64


**Масштабування ознак**

In [51]:
# Нормалізація та навчання моделі:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

X_valid_scaled = scaler.transform(valid_data_processed)

print("Навчальні дані:")
print(X_train_scaled)

print("\n=================\n")
print("Тестові дані:")
print(X_valid_scaled)

Навчальні дані:
[[-0.0218687 -0.0218687  0.        ...  0.         0.         0.       ]
 [-0.0218687 -0.0218687  0.        ...  0.         0.         0.       ]
 [-0.0218687 -0.0218687  0.        ...  0.         0.         0.       ]
 ...
 [-0.0218687 -0.0218687  0.        ...  0.         0.         0.       ]
 [-0.0218687 -0.0218687  0.        ...  0.         0.         0.       ]
 [-0.0218687 -0.0218687  0.        ...  0.         0.         0.       ]]


Тестові дані:
[[-0.0218687 -0.0218687  0.        ...  0.         0.         0.       ]
 [-0.0218687 -0.0218687  0.        ...  0.         0.         0.       ]
 [-0.0218687 -0.0218687  0.        ...  0.         0.         0.       ]
 ...
 [-0.0218687 -0.0218687  0.        ...  0.         0.         0.       ]
 [-0.0218687 -0.0218687  0.        ...  0.         0.         0.       ]
 [-0.0218687 -0.0218687  0.        ...  0.         0.         0.       ]]


**ВИБІР МОДЕЛІ**

**1. Random Forest**

In [52]:
# Ініціалізація та навчання моделі
rendom_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
rendom_forest_model.fit(X_train_scaled, y_train_resampled)

# Передбачення
y_pred = rendom_forest_model.predict(X_test_scaled)

# Обчислення метрик
balanced_acc = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy: {balanced_acc:.2f}")

# Розрахунок метрик і вивід результатів
report = classification_report(y_test, y_pred)
print(report)

Balanced Accuracy: 0.75
              precision    recall  f1-score   support

           0       0.83      0.64      0.72       263
           1       0.70      0.87      0.78       259

    accuracy                           0.75       522
   macro avg       0.77      0.75      0.75       522
weighted avg       0.77      0.75      0.75       522



**2. Support Vector Machine (SVM)**

In [53]:
# Ініціалізація моделі
svc_model = SVC(kernel='linear', random_state=42)

# Навчання моделі
svc_model.fit(X_train_scaled, y_train_resampled)

# Передбачення
y_pred = svc_model.predict(X_test_scaled)

# Обчислення balanced accuracy
balanced_acc = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy: {balanced_acc:.2f}")

# Розрахунок метрик і вивід результатів
report = classification_report(y_test, y_pred)
print(report)

Balanced Accuracy: 0.68
              precision    recall  f1-score   support

           0       0.68      0.69      0.69       263
           1       0.68      0.67      0.68       259

    accuracy                           0.68       522
   macro avg       0.68      0.68      0.68       522
weighted avg       0.68      0.68      0.68       522



**3. Gradient Boosting**

In [54]:
# Ініціалізація моделі
gradient_boosting_model = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Навчання моделі
gradient_boosting_model.fit(X_train_scaled, y_train_resampled)

# Передбачення
y_pred = gradient_boosting_model.predict(X_test_scaled)

# Обчислення balanced accuracy
balanced_acc = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy: {balanced_acc:.2f}")

# Розрахунок метрик і вивід результатів
report = classification_report(y_test, y_pred)
print(report)

Balanced Accuracy: 0.75
              precision    recall  f1-score   support

           0       0.83      0.63      0.72       263
           1       0.70      0.87      0.78       259

    accuracy                           0.75       522
   macro avg       0.77      0.75      0.75       522
weighted avg       0.77      0.75      0.75       522



**ОТРИМАННЯ ТА ЗАПИС У ФАЙЛ ПРОГНОЗУ ДЛЯ ВАЛІДАЦІЙНИХ ДАНИХ**

In [55]:
y_pred = rendom_forest_model.predict(X_valid_scaled)

# Створення DataFrame з результатами
result_df = pd.DataFrame({
    'index': np.arange(len(y_pred)),
    'y': y_pred
})

# Записуємо результати у CSV файл
file_name = 'predictions.csv'
result_df.to_csv(file_name, index=False)
print(f"Прогнози збережені у файл {file_name}")

# файли у робочому каталозі
print("Файли в /kaggle/working:")
print(os.listdir('/kaggle/working'))

Прогнози збережені у файл predictions.csv
Файли в /kaggle/working:
['.virtual_documents', 'predictions.csv']
